# **Extract the Aneurysm Surface**
<hr>

In [1]:
from vmtk import vmtkscripts
import numpy as np
import os

In [2]:
# Change to directory where vmtk_functions and vmtk_filenames are located
os.chdir('/home/iagolessa/Documents/aneurysms/vmtkScripts')

# Import the necessary user's modules
import vmtk_functions
from vmtk_filenames import *

# Change it back
os.chdir('/home/iagolessa')

In [7]:
# Path to save the generated files when extracting the surface
# We reccomend to separate it from the DICOM original dir. 
# Note that inside casePath, this code will create different sub-directories to store different data types
casePath = '/home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case12/'
# Define subdirs
imagesDir       = casePath+'images/'
surfacesDir     = casePath+'surfaces/'
centerlinesDir  = casePath+'centerlines/'
meshesDir       = casePath+'meshes/'
parentVesselDir = casePath+"parentVessel/"

print('Files saved to:', casePath)

Files saved to: /home/iagolessa/Documents/aneurysms/geometries/einsteinCases/unrupturedCases/vmtkReconstruction/case12/


<hr>
# **Extracting the aneurysm surface**

One of the simplest ways of extracting the aneurysm surface is using the *vmtksurfaceregiondrawing* script and draw a line on the aneurysm neck. Of course, this procedure requires the expertise of a physician to correctly iddentify the aneurysm neck. This is perfomed on the next cell: the procedure initially calls the region drawing script, then the field is smoothed and finally the path is cut using the clipper tool with an array. The cell, then, outputs the surface area and aneurysm volume. 

Tests to automatically extract the aneurysm sac are performed below.

In [8]:
aneurysmNeckDraw = vmtkscripts.vmtkSurfaceRegionDrawing()

aneurysmNeckDraw.SurfaceInputFileName = surfacesDir+surfaceOrientedFile
aneurysmNeckDraw.IORead()
aneurysmNeckDraw.Execute()

# Debug
# aneurysmNeckDraw.PrintInputMembers()
# aneurysmNeckDraw.PrintOutputMembers()


# Since the array created by RegionDrawing is discontinous
# we use the array smoothing
aneurysmNeckSmoother = vmtkscripts.vmtkSurfaceArraySmoothing()
aneurysmNeckSmoother.Surface = aneurysmNeckDraw.Surface
aneurysmNeckSmoother.SurfaceArrayName = aneurysmNeckDraw.ContourScalarsArrayName
# General options
aneurysmNeckSmoother.Connexity = 1
aneurysmNeckSmoother.Relaxation = 1.0
aneurysmNeckSmoother.Iterations = 30
aneurysmNeckSmoother.Execute()

# Debug
# aneurysmNeckSmoother.PrintInputMembers()
# aneurysmNeckSmoother.PrintOutputMembers()

# Surface clipper
aneurysmClipper = vmtkscripts.vmtkSurfaceClipper()

aneurysmClipper.Surface = aneurysmNeckSmoother.Surface
# aneurysmClipper.SurfaceInputFileName = surfacesDir+'surfaceWithClipArray.vtp'
# aneurysmClipper.IORead()
# aneurysmClipper.InsideOut = 0
# aneurysmClipper.WidgetType = 'box'
aneurysmClipper.Interactive = 0
aneurysmClipper.ClipArrayName = aneurysmNeckDraw.ContourScalarsArrayName
aneurysmClipper.ClipValue = 0.5*(aneurysmNeckDraw.InsideValue + \
                                 aneurysmNeckDraw.OutsideValue)
aneurysmClipper.Execute()

# Debug
# aneurysmClipper.PrintInputMembers()
# aneurysmClipper.PrintOutputMembers()

# # Extract largest connected surface
# surfaceConnected = vmtkscripts.vmtkSurfaceConnectivity()
# surfaceConnected.Surface = surfaceArrayClipper.Surface
# surfaceConnected.Execute()

# Check if is correct (debug)
vmtk_functions.viewSurface(aneurysmClipper.ClippedSurface)


# Simple remesh procedure to increase quality at cut lines
# aneurysmRemesh.Surface = aneurysmClipper.Surface
# aneurysmRemesh.ElementSizeMode = "edgelength"
# aneurysmRemesh.Execute()


# Write aneurysm surface
vmtk_functions.writeSurface(aneurysmClipper.ClippedSurface,surfacesDir+'patches/aneurysm.stl','binary')

# Calculate volume and area of surface
aneurysmMassProperties = vmtkscripts.vmtkSurfaceMassProperties()
aneurysmMassProperties.Surface = aneurysmClipper.ClippedSurface
aneurysmMassProperties.Execute()

print('Surface area: '+str(aneurysmMassProperties.SurfaceArea)+' mm2')
print('Volume: '+str(aneurysmMassProperties.Volume)+' mm3')

Reading VTK XML surface file.
Quit renderer
Quit renderer
Writing STL surface file.
Surface area: 67.02327634909156 mm2
Volume: 93.64268605434252 mm3


<hr>
<hr>
Extracting the aneurysm surface using the clipper and an array defined on the surface built with the *vmtkdistancetocenterlines* script.
The procedures works better on lateral aneurysms cases; the parent vessel centerlines must be calculated (first part of the script): the reference systems are computed at every inlet and outlet. The surface is then clipped at the level 0 of the DistanceToCenterlines array, calculated using the Tube Function.

The procedure uses the array *capsGeometryArray* constructed above in the reorientation procedure: it uses the information about the inlet patch to construct the parent vessel centerlines automatically.

In [ ]:
# # Function to clip the aneurysm using a spherical clipper widget
# def clipAneurysm(surface):
#     """Function to clip the aneurysm using a spherical clipper widget"""
#     aneurysm = vmtkscripts.vmtkSurfaceClipper()
    
#     aneurysm.Surface = surface
#     aneurysm.InsideOut = 0
#     aneurysm.WidgetType = 'box'
#     aneurysm.Execute()
    
#     return aneurysm.Surface

In [ ]:
# # Reading the input surface
surfaceInput = readSurface(surfacesDir+surface)
viewSurface(surfaceInput)

In [ ]:
# Surface boundary inspector: we need to inspect the boundary of the oriented surface now
# Conveting surface to mesh to get cell entity ids
surfaceOrientedToMesh = vmtkscripts.vmtkSurfaceToMesh()

# Needs to be a .vtp surface file!! not stl
# surfaceOrientedToMesh.Surface = surfaceCapper.Surface #surfaceTransform.Surface
# or read
surfaceOrientedToMesh.SurfaceInputFileName = surfacesDir+surfaceOrientedFile
surfaceOrientedToMesh.IORead()
surfaceOrientedToMesh.Execute()

# Inspecting
surfaceBoundaryInspector = vmtkscripts.vmtkMeshBoundaryInspector()

surfaceBoundaryInspector.Mesh = surfaceOrientedToMesh.Mesh
surfaceBoundaryInspector.CellEntityIdsArrayName = 'CellEntityIds'
surfaceBoundaryInspector.ReferenceSystemsOutputFileName = surfacesDir+referenceSystemsOrientedFile
surfaceBoundaryInspector.Execute()
surfaceBoundaryInspector.IOWrite()

In [ ]:
# Manipulation of the reference system array 
# The code below transforms the referenceSystem.dat info
# to a numpy structured array called 'capsGeometryArray'
# which contains Center, Normals, Radius, Ids and patch type
# of the surface caps

capsGeometryList = []
# Columns to extract
# Center position, normals, radius and ids of caps
# POSSIBLE IMPROVEMENT: GET COLUMNS BY NAME
cols = (0, 1, 2, 3, 4, 5, 6, 13)
colsType = [('Center', tuple),
            ('Normal', tuple),
            ('Radius',float),
            ('Id',int),
            ('PatchType','U10')]

# Get array from referenceSystem.dat file
arrayDatFile = np.genfromtxt(surfacesDir+referenceSystemsOrientedFile,skip_header=1,usecols=cols)
for row in arrayDatFile:
    # Copy formatted to list
    Center = tuple(row[0:3])
    Normal = tuple(row[3:6])
    Radius = row[6]
    Id = row[7]
    capsGeometryList.append((Center, Normal, Radius, Id,'patch'))

# Convert to array
capsOrientedGeoArray = np.array(capsGeometryList, dtype=colsType)
capsOrientedGeoArray

In [ ]:
# Getting the inlet index by searching the patch with largest area

capsArea = np.zeros(len(capsOrientedGeoArray), 
                    dtype=[('Area',float),
                           ('Id',int)])

capsArea['Area'] = (np.pi/4)*capsOrientedGeoArray['Radius']**2
capsArea['Id'] = capsOrientedGeoArray['Id']

for i in range(0, len(capsArea)):
    if capsArea['Area'][i] == np.max(capsArea['Area']):
        inletIndex = capsArea['Id'][i]
        capsOrientedGeoArray['PatchType'][i] = 'inlet'
    else:
        capsOrientedGeoArray['PatchType'][i] = 'outlet'

print('Inlet index (supposing is the cap with larger area) is: ',inletIndex)
capsOrientedGeoArray

In [ ]:
# Specify the other inlets here
extraInletIds = []
# For all the patches ids
for patchId in capsOrientedGeoArray['Id']:
    # If the id is an extraInlet id
    # Check if extraInletIds is not empty
    if extraInletIds:
        # If the patch id is on specified inlet list
        if patchId in extraInletIds:
            # find its index in capsGeometryArray
            index = np.where(capsOrientedGeoArray['Id'] == patchId)
            capsOrientedGeoArray['PatchType'][index[0]] = 'inlet'
        # If not, then name it 'outlet'
        else:
            index = np.where(capsOrientedGeoArray['Id'] == patchId)
            capsOrientedGeoArray['PatchType'][index[0]] = 'outlet'
    else:
        print('No extra inlets!')
        
capsOrientedGeoArray['PatchType']

In [ ]:
# Counting the number of inlets and outlets
# using the numpy.unique function: it returns an ndarray with the unique itens of the input array
# with the return_counts=True also returns the number of appearance of each item
patchTypes, numberOfPatches = np.unique(capsOrientedGeoArray['PatchType'], return_counts=True)

# Zip both patchTypes and numberOfPatches and convert to dictionary
# with number of inlets and outlets
nPatches = dict(zip(patchTypes, numberOfPatches))
nPatches

In [ ]:
# Clears object outletBarycenters if exists
outletBarycenters = None
inletBarycenters = []

# create tuples with inlet and outlet barycenters
for i in range( len(capsOrientedGeoArray) ):
    if capsOrientedGeoArray['PatchType'][i] == 'inlet':
        inletBarycenters.append(capsOrientedGeoArray['Center'][i])
    else:
        # Tye to append outet barycenter, if it cant, then define
        # outletBaraycenters with the first outlet 
        try:
            outletBarycenters += capsOrientedGeoArray['Center'][i]
        except:
            outletBarycenters = capsOrientedGeoArray['Center'][i]
            
inletBarycenters, outletBarycenters

In [ ]:
# inletBarycenters = [(1.491148829460144, 3.6632299423217773, 13.445119857788086)]
# outletBarycenters = (1.625938693905482e-06, -4.1589964894228615e-07, -3.909442057192791e-06,-13.27431869506836, -8.370388984680176, 21.518117904663086,-2.2654712200164795, -8.190746307373047, 24.120887756347656)

In [ ]:
# Computing centerlines
# This code computes centerlines for a generic number of inlets
# It does that by computing the centerlines for each source barycenter
# and then appending the resulting centerlines with 'vmtksurfaceappend'

centerlinesList = []

for source in inletBarycenters:
    # Instantiate vmtkcenterline object
    centerlines = vmtkscripts.vmtkCenterlines()
    # Surface to be used
    centerlines.Surface = surfaceOrientedToMesh.Surface
    # centerlines.SurfaceInputFileName = casePath+'surfaces/patches/wall.stl'
    # centerlines.IORead()
    
    # Type of seed selector: by source coordinate 'pointlist' or 'idlist' by list of ids
    centerlines.SeedSelectorName = 'pointlist'
    centerlines.SourcePoints = source
    centerlines.TargetPoints = list(outletBarycenters)
    centerlines.CheckNonManifold = 1
    centerlines.AppendEndPoints = 1
    centerlines.Execute()
    centerlinesList.append(centerlines.Centerlines)

In [ ]:
# # Instantiate vmtkcenterline object
# centerlines = vmtkscripts.vmtkCenterlines()
# # Surface to be used
# # centerlines.Surface = surfaceOrientedToMesh.Surface
# centerlines.SurfaceInputFileName = surfacesDir+surfaceOrientedFile
# centerlines.IORead()

# # Type of seed selector: by source coordinate 'pointlist' or 'idlist' by list of ids
# #     centerlines.SeedSelectorName = 'pointlist'
# #     centerlines.SourcePoints = # source
# #     centerlines.TargetPoints = [3.8962182998657227, 0.3059086501598358, 19.750572204589844,-2.712946240990277e-07, -4.236111905697726e-08, -5.046152296017681e-07]#list(outletBarycenters)

# centerlines.SeedSelectorName = 'idlist'
# centerlines.SourceIds = [3]
# centerlines.TargetIds = [4,5,6]#list(outletBarycenters)
# centerlines.CheckNonManifold = 1
# centerlines.AppendEndPoints = 1
# centerlines.Execute()
# # centerlinesList.append(centerlines.Centerlines)
# viewCenterline(centerlines.Centerlines,None)

In [ ]:
# Provide managing exception if there is only one centerline
centerlineMain = centerlinesList[0]
centerlineMain

In [ ]:
writeSurface(centerlinesList[0],centerlinesDir+'centerlines.vtp','ascii')

In [ ]:
len(centerlinesList)

In [ ]:
viewCenterline(centerlinesList[0],None)

In [ ]:
if len(centerlinesList) > 1:
    for centerline in centerlinesList[1:]:
        # Instantiate vmtksurfaceappend object
        centerlinesAppend = vmtkscripts.vmtkSurfaceAppend()
        centerlinesAppend.Surface = centerlineMain
        centerlinesAppend.Surface2 = centerline
        centerlinesAppend.Execute()
        centerlineMain = centerlinesAppend.Surface
    centerlinesAppend.SurfaceOutputFileName = centerlinesDir+centerlineWithoutAneurysm
    centerlinesAppend.IOWrite()
else:
    print('Only one source.')

In [ ]:
distanceToCenterlines = vmtkscripts.vmtkDistanceToCenterlines()

# distanceToCenterlines.Surface = surfaceInput
distanceToCenterlines.SurfaceInputFileName = surfacesDir+'patches/wall.stl'
distanceToCenterlines.IORead()

distanceToCenterlines.Centerlines = centerlineMain
distanceToCenterlines.UseRadiusInformation = 1
distanceToCenterlines.EvaluateTubeFunction = 1
distanceToCenterlines.EvaluateCenterlineRadius = 1
distanceToCenterlines.ProjectPointArrays = 1
distanceToCenterlines.SurfaceOutputFileName = surfacesDir+surfaceDistToCenterlinesFile

# Important to remember of the MaximumInscribedSphereRadiusArray !
distanceToCenterlines.RadiusArrayName = centerlines.RadiusArrayName

# Execute distance to centerlines
distanceToCenterlines.Execute()
distanceToCenterlines.IOWrite()

In [ ]:
# Aneurysm clipper
aneurysmClipper = vmtkscripts.vmtkSurfaceClipper()

aneurysmClipper.Surface = distanceToCenterlines.Surface
aneurysmClipper.Interactive = 0
aneurysmClipper.CleanOutput = 0
aneurysmClipper.ClipArrayName = 'DistanceToCenterlines'
aneurysmClipper.ClipValue = 1.9
aneurysmClipper.SurfaceOutputFileName = surfacesDir+'patches/aneurysmClipped.vtp'
aneurysmClipper.ClippedSurfaceOutputFileName = surfacesDir+'patches/surfaceClipped.vtp'
# aneurysmClipper.ClipWidget = 'plane'
# aneurysmClipper.ClipFunction = 1
aneurysmClipper.Execute()
aneurysmClipper.IOWrite()

# If the chosen level is not enough to extract only the aneurysm, then clip the aneurysm
# aneurysm = clipAneurysm(aneurysmClipper.Surface)
viewSurface(aneurysmClipper.Surface)

In [ ]:
aneurysm = clipAneurysm(aneurysmClipper.Surface)

In [ ]:
writeSurface(aneurysmClipper.Surface,surfacesDir+'patches/aneurysmClipped.vtp','ascii')